In [1]:
from tensorflow.keras import layers,models,Model,Sequential
import tensorflow as tf

In [2]:
def AlexNet_v1(im_height=224,im_width=224,class_num=1000):
    # tensorflow中的tensor通道排序是 NHWC
    input_image = layers.Input(shape=(im_height,im_width,3),dtype='float32')                                          # output(None,224,224,3)
    x = layers.ZeroPadding2D(((1,2),(1,2)))(input_image)      # 输入通道的上方补一行0 下方补两行0 左边补一行0 右边补两行0     # output(None,227,227,3)
    x = layers.Conv2D(48,kernel_size=11,strides=4,activation='relu')(x)     # output(None,55,55,48)
    x = layers.MaxPool2D(pool_size=3,strides=2)(x)                           # output(None,27,27,48)
    
    x = layers.Conv2D(128,kernel_size=5,padding='same',activation='relu')(x)  # output(None,27,27,128)
    x = layers.MaxPool2D(pool_size=3,strides=2)(x)                           # output(None,13,13,192)

    x = layers.Conv2D(192,kernel_size=3,padding='same',activation='relu')(x) # output(None,13,13,192)
    x = layers.Conv2D(192,kernel_size=3,padding='same',activation='relu')(x) # output(None,13,13,192)
    x = layers.Conv2D(128,kernel_size=3,padding='same',activation='relu')(x) # output(None,13,13,128)
    x = layers.MaxPool2D(pool_size=3,strides=2)(x)                           # output(None,6,6,128)
    
    x = layers.Flatten()(x)                                                  # output(None,6*6*128)
    x = layers.Dropout(0.2)(x)
    x = layers.Dense(2048,activation='relu')(x)                             # output(None,2048)
    x = layers.Dropout(0.2)(x)
    x = layers.Dense(2048,activation='relu')(x)                             # output(None,2048)
    x = layers.Dense(class_num)(x)
    
    predict = layers.Softmax()(x)
    
    model = models.Model(inputs=input_image,outputs=predict)
    return model

In [3]:
class AlexNet_v2(Model):
    def __init__(self,class_num=1000):
        super(AlexNet_v2,self).__init__()
        self.features = Sequential([
            layers.ZeroPadding2D(((1,2),(1,2))),
            layers.Conv2D(48,kernel_size=11,strides=4,activation='relu'),
            layers.MaxPool2D(pool_size=3,strides=2),
            layers.Conv2D(128,kernel_size=5,padding='same',activation='relu'),
            layers.MaxPool2D(pool_size=3,strides=2),
            layers.Conv2D(192,kernel_size=3,padding='same',activation='relu'),
            layers.Conv2D(192,kernel_size=3,padding='same',activation='relu'),
            layers.Conv2D(128,kernel_size=3,padding='same',activation='relu'),
            layers.Conv2D(pool_size=3,strides=2)
        ])
        self.flatten = layers.Flatten()
        self.classifier = Sequential([
            layers.Dropout(0.2),
            layers.Dense(1024,activation='relu'),
            layers.Dropout(0.2),
            layers.Dense(128,activation='relu'),
            layers.Dense(class_num),
            layers.Softmax()
        ])
        
    def call(self,inputs,**kwargs):
        x = self.features(inputs)
        x = self.flatten(x)
        x = self.classifier(x)
        return x

In [4]:
def AlexNet_v1_2gpu(im_height=224,im_width=224,class_num=1000):
     # tensorflow中的tensor通道排序是 NHWC
    input_image = layers.Input(shape=(im_height,im_width,3),dtype='float32')                                          # output(None,224,224,3)
    x = layers.ZeroPadding2D(((1,2),(1,2)))(input_image)      # 输入通道的上方补一行0 下方补两行0 左边补一行0 右边补两行0     # output(None,227,227,3)
    x = layers.Conv2D(96,kernel_size=11,strides=4,activation='relu')(x)     # output(None,55,55,48)
    x = layers.MaxPool2D(pool_size=3,strides=2)(x)                           # output(None,27,27,48)

    x = layers.Conv2D(256,kernel_size=5,padding='same',activation='relu')(x)  # output(None,27,27,128)
    x = layers.MaxPool2D(pool_size=3,strides=2)(x)                           # output(None,13,13,192)

    x = layers.Conv2D(384,kernel_size=3,padding='same',activation='relu')(x) # output(None,13,13,192)
    x = layers.Conv2D(384,kernel_size=3,padding='same',activation='relu')(x) # output(None,13,13,192)
    x = layers.Conv2D(256,kernel_size=3,padding='same',activation='relu')(x) # output(None,13,13,128)
    x = layers.MaxPool2D(pool_size=3,strides=2)(x)                           # output(None,6,6,128)

    x = layers.Flatten()(x)                                                  # output(None,6*6*128)
    x = layers.Dropout(0.2)(x)
    x = layers.Dense(4096,activation='relu')(x)                             # output(None,2048)
    x = layers.Dropout(0.2)(x)
    x = layers.Dense(4096,activation='relu')(x)                             # output(None,2048)
    x = layers.Dense(class_num)(x)
    
    predict = layers.Softmax()(x)
    
    model = models.Model(inputs=input_image,outputs=predict)
    return model